In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen


/Users/srav88/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [7]:
# Reading CSV File for first Batch 0-5000

match_logs_list_urls = pd.read_csv('match_logs_list_urls.csv')

match_logs_list_urls

,Unnamed: 0,0
0,0,/en/players/b682dcf6/matchlogs/2014-2015/summa...
1,1,/en/players/3564689a/matchlogs/2000-2001/summa...
2,2,/en/players/9823be92/matchlogs/2018-2019/summa...
3,3,/en/players/d0a1ddcb/matchlogs/2015-2016/summa...
4,4,/en/players/9bb0b9f1/matchlogs/2018-2019/summa...
...,...,...
84954,84954,/en/players/f1f92f1c/matchlogs/2016-2017/summa...
84955,84955,/en/players/ee6208fa/matchlogs/1993-1994/summa...
84956,84956,/en/players/464cbbc9/matchlogs/2005-2006/summa...
84957,84957,/en/players/2ac01901/matchlogs/2002-2003/summa...


In [9]:
# This one works

match_logs_list_urls = list(match_logs_list_urls['0'])
match_logs_list_urls = ['https://fbref.com' + i for i in match_logs_list_urls]

In [17]:
match_logs_list_urls[0]
start = match_logs_list_urls[0].find("players/") + len("players/")
end = match_logs_list_urls[0].find("/matchlogs")
substring = match_logs_list_urls[0][start:end]

substring, match_logs_list_urls[0]

('b682dcf6',
 'https://fbref.com/en/players/b682dcf6/matchlogs/2014-2015/summary/Sokratis-Papastathopoulos-Match-Logs')

In [57]:
# Pull all match_log_lists_x tables. We will convert each list individually WORK IN PROCESS

def create_match_logs_tables(match_logs_list_urls_x):

    df_30_columns = pd.DataFrame([])
    df_39_columns = pd.DataFrame([])


    for player in match_logs_list_urls_x:
        try: # this may fix "HTTP Error 404: Not Found"
            # urlopen(player)

            new_table = pd.read_html(player)[0]
            new_table.columns = new_table.columns.droplevel()
            new_table['name'] = player.split('/')[-1].replace("-Match-Logs", "")
            
            if new_table.shape[1] == 30:
                df_30_columns = df_30_columns.append(new_table, ignore_index=True)
                df_30_columns['FBRefID'] = player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))]
                
            if new_table.shape[1] == 39:
                df_39_columns = df_39_columns.append(new_table, ignore_index=True)
                df_39_columns['FBRefID'] = player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))]

        except:
            pass
    

    return df_30_columns, df_39_columns



In [58]:
len(match_logs_list_urls)

84959

In [59]:
# Creating different length data frames
df_30_columns_1, df_39_columns_1 = create_match_logs_tables(match_logs_list_urls[0:1])

In [60]:
df_30_columns_1

,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,Pos,...,Off,Crs,TklW,Int,OG,PKwon,PKcon,Match Report,name,FBRefID
0,2014-08-13,Wed,DFL-Supercup,DFL-Supercup,Home,W 2–0,Dortmund,Bayern Munich,Y,CB,...,0,0,0,5,0,0,0,Match Report,Sokratis-Papastathopoulos,b682dcf6
1,2014-08-16,Sat,DFB-Pokal,First round,Away,W 4–1,Dortmund,S'gart Kickers,Y,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6
2,2014-08-23,Sat,Bundesliga,Matchweek 1,Home,L 0–2,Dortmund,Leverkusen,Y,NaN,...,0,0,2,6,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6
3,2014-08-29,Fri,Bundesliga,Matchweek 2,Away,W 3–2,Dortmund,Augsburg,Y,NaN,...,0,0,2,0,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6
4,2014-09-07,Sun,Euro Qualifying,Group stage,Home,L 0–1,gr Greece,ro Romania,Y,CB,...,0,0,0,3,0,0,0,Match Report,Sokratis-Papastathopoulos,b682dcf6
5,2014-09-13,Sat,Bundesliga,Matchweek 3,Home,W 3–1,Dortmund,Freiburg,Y,NaN,...,0,0,2,1,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6
6,2014-09-16,Tue,Champions Lg,Group stage,Home,W 2–0,de Dortmund,eng Arsenal,Y,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6
7,2014-09-20,Sat,Bundesliga,Matchweek 4,Away,L 0–2,Dortmund,Mainz 05,Y,NaN,...,0,0,2,1,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6
8,2014-09-24,Wed,Bundesliga,Matchweek 5,Home,D 2–2,Dortmund,Stuttgart,Y,NaN,...,0,0,4,0,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6
9,2014-09-27,Sat,Bundesliga,Matchweek 6,Away,L 1–2,Dortmund,Schalke 04,N,NaN,...,0,0,2,0,0,NaN,NaN,Match Report,Sokratis-Papastathopoulos,b682dcf6


In [110]:
# export to google drive as df_30_columns_1 and df_columns_1

pd.DataFrame(df_30_columns_1).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/df_30_columns_1.csv', index=False)
pd.DataFrame(df_39_columns_1).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/df_39_columns_1.csv', index = False)

In [111]:
print(df_39_columns_1.columns)
print(df_30_columns_1.columns)

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
       'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'Prog', 'Carries', 'Prog', 'Succ',
       'Att', 'Match Report', 'name'],
      dtype='object')
Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'Int', 'OG', 'PKwon',
       'PKcon', 'Match Report', 'name'],
      dtype='object')


In [25]:
#Combining Df_30_columns_1 and df_39_columns_1 to dataframe_1

cols = ['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Match Report', 'Int', 'name', 'BFRefID']

df1 = df_39_columns_1
df2 = df_30_columns_1

df_final_1 = df1.merge(df2,how='outer', left_on=cols, right_on=cols)

print(df1.shape)
print(df2.shape)
print(df_final.shape)



KeyError: 'Date'

In [115]:
df_final_1.columns

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
       'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'Prog', 'Carries', 'Prog', 'Succ',
       'Att', 'Match Report', 'name', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG',
       'PKwon', 'PKcon'],
      dtype='object')

In [116]:
# export to google drive as dataframe_1

pd.DataFrame(df_final_1).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/df_final_1.csv', index=False)